# Example 3: Decorators - Automatic Function Logging

This notebook demonstrates automatic function logging using decorators with proper console output:

- **@log_call decorator** - Automatically log function calls
- **Logger + ConsoleHandler** - Proper setup for console output
- **LogEntry.create()** - Creating structured log entries
- **Custom Extractors** - Domain-specific context extraction
- **Real-World Examples** - API endpoints, data processing, async functions


## Setup - Imports


In [1]:
import asyncio
import time
from dc_logger.client.Log import LogEntry, LogLevel, LogEntity, HTTPDetails
from dc_logger.client.base import Logger, HandlerInstance, Handler_BufferSettings
from dc_logger.services.console.base import ConsoleHandler, Console_ServiceConfig
from dc_logger.client.decorators import log_call, LogDecoratorConfig
from dc_logger.client.extractors import (
    KwargsEntityExtractor,
    KwargsHTTPDetailsExtractor,
    EntityExtractor
)

print("All imports successful!")
print("Ready to demonstrate @log_call decorator with console output")


All imports successful!
Ready to demonstrate @log_call decorator with console output


---
## Part 1: Setup Logger with ConsoleHandler

First, we set up a Logger with ConsoleHandler to see our logs in real-time.


In [2]:
print("=" * 70)
print("Setting up Logger with ConsoleHandler")
print("=" * 70)

# Step 1: Create Console Service Config
console_config = Console_ServiceConfig(
    output_mode="console",
    output_type="json"  # "text" or "json"
)

# Step 2: Create Console Handler with buffer settings
buffer_settings = Handler_BufferSettings()
console_handler = ConsoleHandler(
    buffer_settings=buffer_settings,
    service_config=console_config
)

# Step 3: Create HandlerInstance (concrete class that wraps ServiceHandler)
# HandlerInstance adds filtering logic (log_level and log_method)
handler_instance = HandlerInstance(
    service_handler=console_handler,
    handler_name="console_text",
    log_level=LogLevel.INFO,
    log_method=["POST", "PUT", "DELETE", "PATCH", "COMMENT", "GET"]
)

# Step 4: Create Logger with the handler
demo_logger = Logger(handlers=[handler_instance])

print("\nLogger configured with ConsoleHandler!")
print(f"  - Handler Name: {handler_instance.handler_name}")
print(f"  - Log Level: {handler_instance.log_level}")
print(f"  - Output Type: {console_config.output_type}")
print(f"  - Output Mode: {console_config.output_mode}")
print("\nArchitecture:")
print("  Logger → HandlerInstance (filtering) → ConsoleHandler (writes to console)")

# Function to get the logger (for decorator)
def get_logger():
    return demo_logger


Setting up Logger with ConsoleHandler

Logger configured with ConsoleHandler!
  - Handler Name: console_text
  - Log Level: LogLevel.INFO
  - Output Type: json
  - Output Mode: console

Architecture:
  Logger → HandlerInstance (filtering) → ConsoleHandler (writes to console)


---
## Part 2: Manual Logging - Creating LogEntry Objects

Before using decorators, let's see how to manually create and log entries.


In [3]:
print("=" * 70)
print("Manual LogEntry Creation")
print("=" * 70)

# Example 1: Simple log entry
log1 = LogEntry.create(
    level=LogLevel.INFO,
    message="Application started",
    app_name="demo_app"
)

print("\nLog Entry 1 Created:")
print(f"  Message: {log1.message}")
print(f"  Level: {log1.level}")
print(f"  App: {log1.app_name}")

# Example 2: Log with entity
log2 = LogEntry.create(
    level=LogLevel.INFO,
    message="Dataset updated successfully",
    app_name="demo_app",
    entity={"type": "dataset", "id": "ds_123", "name": "Sales Data"},
    user="user_456",
    duration_ms=250
)

print("\nLog Entry 2 Created with Entity:")
print(f"  Message: {log2.message}")
print(f"  Entity: {log2.entity.type}/{log2.entity.id}")
print(f"  User: {log2.user}")
print(f"  Duration: {log2.duration_ms}ms")

# Now write these logs using the logger
print("\n" + "=" * 70)
print("Writing logs to console via Logger.write()")
print("=" * 70)

await demo_logger.write(log1)
print()
await demo_logger.write(log2)


Manual LogEntry Creation

Log Entry 1 Created:
  Message: Application started
  Level: LogLevel.INFO
  App: demo_app

Log Entry 2 Created with Entity:
  Message: Dataset updated successfully
  Entity: dataset/ds_123
  User: user_456
  Duration: 250ms

Writing logs to console via Logger.write()
{
  "timestamp": "2025-10-10T12:39:43.897982Z",
  "level": "INFO",
  "app_name": "demo_app",
  "message": "Application started",
  "method": "COMMENT",
  "status": "info",
  "extra": {}
}

{
  "timestamp": "2025-10-10T12:39:43.898205Z",
  "level": "INFO",
  "app_name": "demo_app",
  "message": "Dataset updated successfully",
  "user": "user_456",
  "method": "COMMENT",
  "status": "info",
  "duration_ms": 250,
  "entity": {
    "type": "dataset",
    "id": "ds_123",
    "name": "Sales Data",
    "additional_info": {}
  },
  "extra": {}
}


---
## Part 3: Basic Decorator Usage - @log_call

The `@log_call` decorator automatically creates LogEntry objects and writes them to the logger.


In [4]:
print("=" * 70)
print("Basic @log_call Decorator")
print("=" * 70)

# Example 1: Simplest usage
@log_call(logger_getter=get_logger)
def add_numbers(a, b):
    """Add two numbers."""
    return a + b

print("\nCalling add_numbers(5, 3)...")
result1 = add_numbers(5, 3)
print(f"Result: {result1}")

# Example 2: Function with processing time
@log_call(logger_getter=get_logger)
def process_order(order_id, user_id, total):
    """Process an order."""
    time.sleep(0.1)  # Simulate processing
    return {"status": "completed", "order_id": order_id, "total": total}

print("\n\nCalling process_order...")
result2 = process_order("ORD_123", "user_456", 99.99)
print(f"Result: {result2}")

print("\n\nThe decorator automatically:")
print("  1. Captured function start time")
print("  2. Executed the function")
print("  3. Calculated duration")
print("  4. Created LogEntry with all context")
print("  5. Called logger.write(entry) to send to ConsoleHandler")


Basic @log_call Decorator

Calling add_numbers(5, 3)...
Result: 8


Calling process_order...
Result: {'status': 'completed', 'order_id': 'ORD_123', 'total': 99.99}


The decorator automatically:
  1. Captured function start time
  2. Executed the function
  3. Calculated duration
  4. Created LogEntry with all context
  5. Called logger.write(entry) to send to ConsoleHandler


{
  "timestamp": "2025-10-10T12:39:43.906306Z",
  "level": "INFO",
  "app_name": "default",
  "message": "add_numbers completed",
  "method": "COMMENT",
  "action": "add_numbers",
  "status": "success",
  "duration_ms": 0,
  "correlation": {
    "trace_id": "83044475-f69f-47f1-b121-5989d9b1f54e",
    "span_id": "4253a2dbe2ae493b",
    "parent_span_id": null
  },
  "extra": {}
}
{
  "timestamp": "2025-10-10T12:39:44.006861Z",
  "level": "INFO",
  "app_name": "default",
  "message": "process_order completed",
  "method": "COMMENT",
  "action": "process_order",
  "status": "success",
  "duration_ms": 100,
  "correlation": {
    "trace_id": "83044475-f69f-47f1-b121-5989d9b1f54e",
    "span_id": "da60bf50667d4452",
    "parent_span_id": "4253a2dbe2ae493b"
  },
  "extra": {}
}


---
## Part 4: Using Extractors - Auto-extract Entity Information

Extractors pull context from function arguments to create rich LogEntry objects.


In [5]:
print("=" * 70)
print("Decorator with Entity Extractor")
print("=" * 70)

# Configure decorator with entity extractor
config_with_entity = LogDecoratorConfig(
    entity_extractor=KwargsEntityExtractor(kwarg_name="entity")
)

@log_call(logger_getter=get_logger, config=config_with_entity)
def update_dataset(dataset_id, changes, entity=None):
    """Update a dataset."""
    return {"updated": dataset_id, "changes_applied": len(changes)}

print("\nCalling update_dataset with entity info...")
result = update_dataset(
    dataset_id="ds_sales_2024",
    changes={"columns": ["new_col"]},
    entity={"type": "dataset", "id": "ds_sales_2024", "name": "Sales 2024"}
)

print(f"\nResult: {result}")
print("\nKwargsEntityExtractor automatically:")
print("  - Extracted entity from 'entity' kwarg")
print("  - Created LogEntity object")
print("  - Included it in the LogEntry")
print("  - Logger wrote it to ConsoleHandler")


Decorator with Entity Extractor

Calling update_dataset with entity info...

Result: {'updated': 'ds_sales_2024', 'changes_applied': 1}

KwargsEntityExtractor automatically:
  - Extracted entity from 'entity' kwarg
  - Created LogEntity object
  - Included it in the LogEntry
  - Logger wrote it to ConsoleHandler


{
  "timestamp": "2025-10-10T12:39:44.014971Z",
  "level": "INFO",
  "app_name": "default",
  "message": "update_dataset completed",
  "method": "COMMENT",
  "action": "update_dataset",
  "status": "success",
  "duration_ms": 0,
  "entity": {
    "type": "dataset",
    "id": "ds_sales_2024",
    "name": "Sales 2024",
    "additional_info": {}
  },
  "correlation": {
    "trace_id": "83044475-f69f-47f1-b121-5989d9b1f54e",
    "span_id": "7215fe11e64f4751",
    "parent_span_id": "da60bf50667d4452"
  },
  "extra": {}
}


---
## Part 5: Custom Extractors - Domain-Specific Logic

Create custom extractors for your application's specific needs.


In [6]:
print("=" * 70)
print("Custom Extractor - E-Commerce Orders")
print("=" * 70)

# Define custom extractor for orders
class OrderExtractor(EntityExtractor):
    """Extract order information from function arguments."""
    
    def extract(self, func, args, kwargs):
        if "order_id" in kwargs:
            return LogEntity(
                type="order",
                id=kwargs["order_id"],
                name=f"Order {kwargs['order_id']}",
                additional_info={
                    "customer": kwargs.get("customer_id"),
                    "total": kwargs.get("total")
                }
            )
        return None

# Configure with custom extractor
order_config = LogDecoratorConfig(
    entity_extractor=OrderExtractor()
)

@log_call(logger_getter=get_logger, config=order_config)
def process_payment(order_id, customer_id, total, payment_method):
    """Process payment for an order."""
    time.sleep(0.05)  # Simulate payment processing
    return {
        "transaction_id": "txn_abc123",
        "status": "success",
        "amount": total
    }

print("\nCalling process_payment...")
result = process_payment(
    order_id="ORD_2024_555",
    customer_id="CUST_123",
    total=299.99,
    payment_method="credit_card"
)

print(f"\nResult: {result}")
print("\nCustom OrderExtractor:")
print("  - Automatically detected order_id in kwargs")
print("  - Extracted customer_id and total")
print("  - Created LogEntity with type='order'")
print("  - Included all info in additional_info dict")


Custom Extractor - E-Commerce Orders

Calling process_payment...



Result: {'transaction_id': 'txn_abc123', 'status': 'success', 'amount': 299.99}

Custom OrderExtractor:
  - Automatically detected order_id in kwargs
  - Extracted customer_id and total
  - Created LogEntity with type='order'
  - Included all info in additional_info dict


{
  "timestamp": "2025-10-10T12:39:44.073985Z",
  "level": "INFO",
  "app_name": "default",
  "message": "process_payment completed",
  "method": "COMMENT",
  "action": "process_payment",
  "status": "success",
  "duration_ms": 51,
  "entity": {
    "type": "order",
    "id": "ORD_2024_555",
    "name": "Order ORD_2024_555",
    "additional_info": {
      "customer": "CUST_123",
      "total": 299.99
    }
  },
  "correlation": {
    "trace_id": "83044475-f69f-47f1-b121-5989d9b1f54e",
    "span_id": "74430710bd834070",
    "parent_span_id": "7215fe11e64f4751"
  },
  "extra": {}
}


---
## Part 6: Async Function Support

The @log_call decorator works seamlessly with async functions.


In [7]:
print("=" * 70)
print("Async Function with @log_call")
print("=" * 70)

# Async function with decorator
@log_call(logger_getter=get_logger)
async def fetch_user_data(user_id):
    """Fetch user data asynchronously."""
    await asyncio.sleep(0.1)  # Simulate async I/O
    return {
        "user_id": user_id,
        "name": "Alice Smith",
        "email": "alice@example.com"
    }

print("\nCalling async fetch_user_data...")
result = await fetch_user_data("user_789")
print(f"\nResult: {result}")

# Multiple async calls
@log_call(logger_getter=get_logger)
async def process_batch(items):
    """Process items in batch."""
    await asyncio.sleep(0.05)
    return {"processed": len(items), "status": "complete"}

print("\n\nRunning 3 async operations concurrently...")
tasks = [
    process_batch([1, 2, 3]),
    process_batch([4, 5, 6]),
    process_batch([7, 8, 9])
]
results = await asyncio.gather(*tasks)

print(f"\nBatch Results: {results}")
print("\nEach async call was logged independently with its own duration!")


Async Function with @log_call

Calling async fetch_user_data...
{
  "timestamp": "2025-10-10T12:39:44.197536Z",
  "level": "INFO",
  "app_name": "default_app",
  "message": "fetch_user_data completed",
  "method": "COMMENT",
  "action": "fetch_user_data",
  "status": "success",
  "duration_ms": 114,
  "correlation": {
    "trace_id": "83044475-f69f-47f1-b121-5989d9b1f54e",
    "span_id": "0fec411756424c23",
    "parent_span_id": "74430710bd834070"
  },
  "extra": {
    "function": "fetch_user_data",
    "module": "__main__",
    "caller": {
      "file": "C:\\Users\\Hector\\AppData\\Local\\Temp\\ipykernel_25556\\3701582010.py",
      "line": 17,
      "function": "<module>"
    }
  }
}

Result: {'user_id': 'user_789', 'name': 'Alice Smith', 'email': 'alice@example.com'}


Running 3 async operations concurrently...
{
  "timestamp": "2025-10-10T12:39:44.260077Z",
  "level": "INFO",
  "app_name": "default_app",
  "message": "process_batch completed",
  "method": "COMMENT",
  "action": "pr

---
## Summary

1. **Logger Setup** - Logger with ConsoleHandler for real-time output
2. **LogEntry.create()** - Manual creation of structured log entries
3. **Logger.write()** - Logger's single responsibility: write entries to handlers
4. **@log_call Decorator** - Automatically creates LogEntry and calls logger.write()
5. **Extractors** - Pull context from function arguments (entity, HTTP, multi-tenant)
6. **Custom Extractors** - Create domain-specific extractors for your app
7. **Async Support** - Works seamlessly with async/await functions

### Architecture Flow

```
@log_call decorator wraps function
        ↓
Function executes (timing tracked)
        ↓
Extractors pull context from args/kwargs
        ↓
LogEntry.create() with all context
        ↓
Logger.write(entry)
        ↓
HandlerInstance filters by level/method
        ↓
ConsoleHandler writes to console
```

### Logger Class Responsibilities

**Single Responsibility**: Write LogEntry objects to handlers

```python
class Logger:
    handlers: List[HandlerInstance]
    
    async def write(self, entry: LogEntry):
        for handler in self.handlers:
            await handler.write(entry)
```

### Key Components

- **LogEntry**: Data structure holding log information
- **Logger**: Routes entries to handlers
- **Handler**: Determines where/how to write logs (Console, File, Cloud)
- **Extractor**: Pulls context from function arguments
- **Decorator**: Automates logging without boilerplate
